#### LSTM Baseline

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import warnings
warnings.simplefilter('ignore')

df = pd.read_csv("train.csv") 
df_test = pd.read_csv("test.csv") 

In [9]:
le = LabelEncoder()

df["Country"] = le.fit_transform(df["Country"])
df["City"] = le.fit_transform(df["City"])

In [10]:
# 目的変数と説明変数の定義
X_data = df.drop(columns=['pm25_mid'])
y_data = df['pm25_mid']

# train_test_splitのインポート
from sklearn.model_selection import train_test_split

# shuffle=Falseとすることで時系列が混ざるのを防止

# 学習データおよび検証データと、評価データに80:20の割合で2分割する
X_trainval, X_test, y_trainval, y_test = train_test_split(X_data, y_data, test_size=0.20, shuffle=False)

# 学習データと検証データに75:25の割合で2分割する
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, shuffle=False)

print("finish")

finish


In [17]:
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm

# 関数get_standardized_tの定義
def get_standardized_t(X, num_date):
    X = np.array(X)
    X_t_list = []
    for i in tqdm(range(len(X) - num_date + 1)):
        X_t = X[i:i+num_date]
        scaler = StandardScaler()
        X_standardized = scaler.fit_transform(X_t)
        X_t_list.append(X_standardized)
    return np.array(X_t_list)

num_date = 10

# 学習用、検証用、評価用データの加工
X_train_t =  get_standardized_t(X_train, num_date=num_date)
X_val_t = get_standardized_t(X_val, num_date=num_date)
X_test_t = get_standardized_t(X_test, num_date=num_date)
#df_test = get_standardized_t(df_test, num_date=num_date)


print(X_train_t.shape)
print(X_val_t.shape)
print(X_test_t.shape)
#print(df_test.shape)

100%|██████████| 6/6 [00:00<00:00, 3008.11it/s]

(35, 10, 52)
(6, 10, 52)
(6, 10, 52)


In [18]:
# 目的変数の変形

y_train_t = y_train[num_date-1:]
y_val_t = y_val[num_date-1:]
y_test_t = y_test[num_date-1:]


In [19]:
#Sequentialのインポート
from keras.models import Sequential
# Dense、LSTMのインポート
from keras.layers import Dense, LSTM
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# ネットワークの各層のサイズの定義
num_l1 = 100
num_l2 = 20
num_output = 1

# 以下、ネットワークを構築
model = Sequential()
# 第1層
model.add(LSTM(units=num_l1,
                activation='relu',
                batch_input_shape=(None, X_train_t.shape[1], X_train_t.shape[2])))
# 第2層
model.add(Dense(num_l2, activation='relu'))
# 出力層
model.add(Dense(num_output, activation='linear'))
# ネットワークのコンパイル
opt = tf.keras.optimizers.Adam( learning_rate=0.01 )
#model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.compile(loss=root_mean_squared_error, optimizer="Adam") #metrics=['accuracy'])

# モデルの学習の実行（学習の完了までには数秒から数十秒ほど時間がかかります。）
result = model.fit(x=X_train_t, y=y_train_t, epochs=200, batch_size=512, validation_data=(X_val_t, y_val_t))

# 評価データの正解率の計算
#loss, accuracy = model.evaluate( X_test_t, y_test_t)

# 評価データの正解率の表示
print()
#print('accuracy:', accuracy)

pred = model.predict(X_test_t)

print("RMSE:",np.sqrt(mean_squared_error(y_test_t, pred)))

Epoch 1/200
1/1 [==============================] - 1s 774ms/step - loss: 64.8673 - val_loss: 55.4895
Epoch 2/200
1/1 [==============================] - 0s 85ms/step - loss: 64.7806 - val_loss: 55.4308
Epoch 3/200
1/1 [==============================] - 0s 83ms/step - loss: 64.7214 - val_loss: 55.3634
Epoch 4/200
1/1 [==============================] - 0s 84ms/step - loss: 64.6614 - val_loss: 55.2861
Epoch 5/200
1/1 [==============================] - 0s 81ms/step - loss: 64.5905 - val_loss: 55.1939
Epoch 6/200
1/1 [==============================] - 0s 87ms/step - loss: 64.5039 - val_loss: 55.0880
Epoch 7/200
1/1 [==============================] - 0s 45ms/step - loss: 64.3954 - val_loss: 54.9649
Epoch 8/200
1/1 [==============================] - 0s 51ms/step - loss: 64.2670 - val_loss: 54.8159
Epoch 9/200
1/1 [==============================] - 0s 50ms/step - loss: 64.1131 - val_loss: 54.6332
Epoch 10/200
1/1 [==============================] - 0s 59ms/step - loss: 63.9258 - val_loss: 54.410

Epoch 81/200
1/1 [==============================] - 0s 47ms/step - loss: 6.2061 - val_loss: 19.6521
Epoch 82/200
1/1 [==============================] - 0s 47ms/step - loss: 6.0318 - val_loss: 20.1947
Epoch 83/200
1/1 [==============================] - 0s 53ms/step - loss: 5.8256 - val_loss: 19.4365
Epoch 84/200
1/1 [==============================] - 0s 50ms/step - loss: 5.5198 - val_loss: 19.5650
Epoch 85/200
1/1 [==============================] - 0s 48ms/step - loss: 5.2325 - val_loss: 19.4190
Epoch 86/200
1/1 [==============================] - 0s 47ms/step - loss: 4.9879 - val_loss: 18.8078
Epoch 87/200
1/1 [==============================] - 0s 47ms/step - loss: 4.8092 - val_loss: 19.5951
Epoch 88/200
1/1 [==============================] - 0s 50ms/step - loss: 4.9131 - val_loss: 17.8134
Epoch 89/200
1/1 [==============================] - 0s 48ms/step - loss: 5.6309 - val_loss: 20.4108
Epoch 90/200
1/1 [==============================] - 0s 49ms/step - loss: 6.1098 - val_loss: 18.7489


In [ ]:
import itertools
sample = pd.read_csv("submit_sample.csv",header=None) 

pred = model.predict(df_test)

pred = list(itertools.chain.from_iterable(pred))
pred_df = pd.Series(pred)

number = (num_date + n) - 1
num = np.zeros(number)
pred_num = pd.Series(num)

for i in range(number):
    pred_num[i] = pred_df[number+i]

prediction = pd.concat([pred_num, pred_df], axis=0).reset_index(drop=True)
prediction = pd.DataFrame(prediction, columns=["pred"])

prediction["id"] = df_test_id["id"]
prediction = prediction.sort_values(['id']).reset_index(drop=True)
sample[1] = prediction["pred"]
sample.to_csv("LSTM_submit.csv",index=None,header=None)
sample